In [1]:
import numpy as np
import pandas as pd

In [2]:
db = pd.read_csv('C:/Users/singh/Downloads/smol.csv')

In [3]:
db.columns

Index(['Make', 'Model', 'Index', 'RawData', 'ScaleFactor', 'NumSamples',
       'Duration', 'Fs', 'CroppedData_1', 'CroppedData_2',
       ...
       'CroppedData_999993', 'CroppedData_999994', 'CroppedData_999995',
       'CroppedData_999996', 'CroppedData_999997', 'CroppedData_999998',
       'CroppedData_999999', 'CroppedData_1000000', 'CroppedData_1000001',
       'Features'],
      dtype='object', length=1000010)

In [3]:
bd = np.transpose(db)
column_names = ['DJI1', 'DJI2', 'DJI3', 'DJI4', 'DJI5', 'DJI6','DJI7', 'DJI8', 'DJI9', 'DJI10', 'DJI11', 'DJI12', 'DJI13', 'DJI14', 'DJI15', 'DJI16', 'DJI17']
bd.columns = column_names
bd = bd.drop(index='Make')
bd = bd.drop(index='Model')
bd = bd.drop(index='RawData')
bd = bd.drop(index='NumSamples')
bd = bd.drop(index='Duration')
bd = bd.drop(index='Fs')
bd = bd.drop(index='Features')
bd = bd.drop(index='ScaleFactor')

In [5]:
print(bd)

                      DJI1  DJI2    DJI3   DJI4    DJI5   DJI6  DJI7  DJI8  \
Index                    1     1       1      1       1      1     1     1   
CroppedData_1          310   291      -9    288   -1054    274  -220   200   
CroppedData_2          206   107      27    584    -602     18    -8   164   
CroppedData_3          -26  -137     391    460     -22     78    52  -172   
CroppedData_4         -186  -253     463     24     -90    282    64  -240   
...                    ...   ...     ...    ...     ...    ...   ...   ...   
CroppedData_999997   -4194     7  -10153   -516  -19994  -3982  -424  -476   
CroppedData_999998   -1882  -305  -10213  -1596  -14502   -142  1824  6244   
CroppedData_999999    1462  -849   -4825  -1676    1594   3774  2872  9124   
CroppedData_1000000   3894  -861    2875   -812   13394   5550  2380  6568   
CroppedData_1000001   3862  -269    9399    356   16506   4370   684   432   

                      DJI9   DJI10  DJI11   DJI12  DJI13   DJI1

In [4]:
def haar_wavelet_transform(signal):
    n = len(signal)
    output = np.zeros(n)
    temp = np.zeros(n)
    h = 1/np.sqrt(2)
    for i in range(0, n - 1, 2):
        output[i // 2] = h * (signal[i] + signal[i + 1])
        temp[i // 2] = h * (signal[i] - signal[i + 1])

    return output, temp

data = bd;
decomposition_results = {}
column_names = ['DJI{}'.format(i) for i in range(1, 18)]

for column_name in column_names:
    drone_data = data[column_name].values.astype(float)
    detail_coeffs = drone_data
    approxs = []
    for stage in range(3):
        approx_coeffs, detail_coeffs = haar_wavelet_transform(detail_coeffs)
        approxs.append(approx_coeffs)

    decomposition_results[column_name] = {
        'detail_coefficients': detail_coeffs,
        'approx_coefficients_stage_1': approxs[0],
        'approx_coefficients_stage_2': approxs[1],
        'approx_coefficients_stage_3': approxs[2]
    }
filtered_data = detail_coeffs

In [5]:
decomposition_results['DJI1']['approx_coefficients_stage_3']

array([-322.79424561,  -36.76955262,  138.59292911, ...,    0.        ,
          0.        ,    0.        ])

In [6]:
data = bd['DJI1']
T1 = 3*(np.std(data, axis=0))
T2 = -3*(np.std(data, axis=0))

BELOW_T1 = 0
BETWEEN_T1_T2 = 1
ABOVE_T2 = 2

def map_to_state(value, T1, T2):
    if value < T1:
        return BELOW_T1
    elif value < T2:
        return BETWEEN_T1_T2
    else:
        return ABOVE_T2

states = [map_to_state(value, T1, T2) for value in data]

In [6]:
print(data)

Index                      1
CroppedData_1            310
CroppedData_2            206
CroppedData_3            -26
CroppedData_4           -186
                       ...  
CroppedData_999997     -4194
CroppedData_999998     -1882
CroppedData_999999      1462
CroppedData_1000000     3894
CroppedData_1000001     3862
Name: DJI1, Length: 1000002, dtype: object


In [7]:
print(type(data))

<class 'pandas.core.series.Series'>


In [7]:
# data = data.drop(index = 'detail_coeffs')
print(data)

Index                      1
CroppedData_1            310
CroppedData_2            206
CroppedData_3            -26
CroppedData_4           -186
                       ...  
CroppedData_999997     -4194
CroppedData_999998     -1882
CroppedData_999999      1462
CroppedData_1000000     3894
CroppedData_1000001     3862
Name: DJI1, Length: 1000002, dtype: object


In [8]:
data = data.to_frame()
data['detail_coeffs'] = decomposition_results['DJI1']['detail_coefficients']
print(data)

                      DJI1  detail_coeffs
Index                    1     -59.750523
CroppedData_1          310     -98.994949
CroppedData_2          206      65.053824
CroppedData_3          -26    -224.859956
CroppedData_4         -186     209.303607
...                    ...            ...
CroppedData_999997   -4194       0.000000
CroppedData_999998   -1882       0.000000
CroppedData_999999    1462       0.000000
CroppedData_1000000   3894       0.000000
CroppedData_1000001   3862       0.000000

[1000002 rows x 2 columns]


In [9]:
# data = data.drop('DJI1', axis=1)
data['label'] = data['detail_coeffs'].apply(lambda x: 1 if x != 0 else 0)
print(data)

                      DJI1  detail_coeffs  label
Index                    1     -59.750523      1
CroppedData_1          310     -98.994949      1
CroppedData_2          206      65.053824      1
CroppedData_3          -26    -224.859956      1
CroppedData_4         -186     209.303607      1
...                    ...            ...    ...
CroppedData_999997   -4194       0.000000      0
CroppedData_999998   -1882       0.000000      0
CroppedData_999999    1462       0.000000      0
CroppedData_1000000   3894       0.000000      0
CroppedData_1000001   3862       0.000000      0

[1000002 rows x 3 columns]


In [10]:
yT_uav = data[data['label'] == 1]['detail_coeffs'].values
yT_noise = data[data['label'] == 0]['detail_coeffs'].values

In [11]:
labels = data['label']
TN = np.zeros((3, 3), dtype=int)

def calculate_transition_matrix(yT):
    states = np.zeros(len(yT), dtype=int)
    for i, value in enumerate(yT):
        if value > T1:
            states[i] = 0  
        elif T2 <= value <= T1:
            states[i] = 1  
        else:
            states[i] = 2  
            
    transition_counts = np.zeros((3, 3), dtype=int)
    for i in range(1, len(states)):
        prev_state = states[i - 1]
        current_state = states[i]
        transition_counts[prev_state, current_state] += 1
        TN[prev_state, states] += 1


    transition_prob_matrix = np.zeros((3, 3))
    for i in range(3):
        total_transitions_from_state = np.sum(transition_counts[i])
        if total_transitions_from_state > 0:
            transition_prob_matrix[i] = transition_counts[i] / total_transitions_from_state

#     transition_prob = np.nan_to_num(transition_counts / transition_count.sum(axis=1, keepdims=True))
    return transition_prob_matrix

In [13]:
transition_matrix_uav = calculate_transition_matrix(yT_uav)
print("Transition Probability Matrix for UAV:")
print(transition_matrix_uav)

Transition Probability Matrix for UAV:
[[6.55172414e-01 3.44827586e-01 0.00000000e+00]
 [8.01410482e-05 9.99847732e-01 7.21269434e-05]
 [0.00000000e+00 3.33333333e-01 6.66666667e-01]]


In [14]:
transition_matrix_noise = calculate_transition_matrix(yT_noise)
print("Transition Probability Matrix for Noise:")
print(transition_matrix_noise)

Transition Probability Matrix for Noise:
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


In [15]:
print("Transition Number Matrix for Test Signal:\n", TN)

Transition Number Matrix for Test Signal:
 [[     58      58      58]
 [ 249560 1876750  249560]
 [     54      54      54]]


In [17]:
def calculate_likelihood(yT, transition_prob_matrix, tn_matrix):
    likelihood = 1.0
    for i in range(3):
        for j in range(3):
            likelihood *= transition_prob_matrix[i, j] ** tn_matrix[i, j]
    return likelihood

def calculate_posterior(yT, tp_uav, tp_noise, tn_matrix, prior_uav=0.5, prior_noise=0.5):
    likelihood_uav = calculate_likelihood(yT, tp_uav, tn_matrix)
    likelihood_noise = calculate_likelihood(yT, tp_noise, tn_matrix)
    
    evidence = (likelihood_uav * prior_uav) + (likelihood_noise * prior_noise)
    
    posterior_uav = (likelihood_uav * prior_uav) / evidence
    posterior_noise = (likelihood_noise * prior_noise) / evidence
    
    return posterior_uav, posterior_noise

posterior_uav, posterior_noise = calculate_posterior(data, transition_matrix_uav, transition_matrix_noise, TN)

if posterior_uav >= posterior_noise:
    print("Classified as UAV signal")
else:
    print("Classified as noise")


Classified as noise


C:\Users\singh\AppData\Local\Temp\ipykernel_15156\3543434879.py:14: RuntimeWarning: invalid value encountered in double_scalars
  posterior_uav = (likelihood_uav * prior_uav) / evidence
C:\Users\singh\AppData\Local\Temp\ipykernel_15156\3543434879.py:15: RuntimeWarning: invalid value encountered in double_scalars
  posterior_noise = (likelihood_noise * prior_noise) / evidence


In [ ]:
values = data.values
yT = data

def calculate_transition_matrix(states):
    TN = np.zeros((3, 3))
    for i in range(len(states) - 1):
        TN[states[i], states[i+1]] += 1
    total_transitions = np.sum(TN)
    TP = TN / total_transitions
    return TP
noise_signals = yT_noise  

all_noise_values = []
for yT in yT_noise:
    all_noise_values.extend(yT)
sigma = np.std(all_noise_values)

uav_signals = yT_uav  

all_uav_states = []
for yT in yT_uav:
    all_uav_states.extend(calculate_states(yT, T1, T2))
tp_uav = calculate_transition_matrix(all_uav_states)

all_noise_states = []
for yT in yT_noise:
    all_noise_states.extend(calculate_states(yT, T1, T2))
tp_noise = calculate_transition_matrix(all_noise_states)

In [ ]:
likelihood_C1 = np.prod(np.power(transition_matrix_uav, TN))
likelihood_C0 = np.prod(np.power(transition_matrix_noise, TN))


likelihood_C1 = 1.0
for i in range(3):
    for j in range(3):
        likelihood_C1 *= transition_matrix_uav[i, j] ** TN[i, j]

likelihood_C0 = 1.0
for i in range(3):
    for j in range(3):
        likelihood_C0 *= transition_matrix_noise[i, j] ** TN[i, j]

print("P(yT | C = 1) =", likelihood_C1)
print("P(yT | C = 0) =", likelihood_C0)